# Get Started with Earth Engine

This Get Started guide is intended as a quick way to start programming with the Earth Engine Python API. For an introductory look at Python and more in-depth exercises with the Earth Engine API, see the [tutorials](https://github.com/giswqs/earthengine-py-notebooks). For suggestions on Python coding style, see the [Google Python Style Guide](http://google.github.io/styleguide/pyguide.html).[link text](https://)

Google Earth Engine allows users to run algorithms on georeferenced imagery and vectors stored on Google's infrastructure. The Google Earth Engine API provides a library of functions which may be applied to data for display and analysis. Earth Engine's [public data catalog](https://developers.google.com/earth-engine/datasets/) contains a large amount of publicly available imagery and vector datasets. Private assets can also be created in users' personal folders.


## Installing the Earth Engine API

In [1]:
# Installs geemap package
# !pip install geemap

import geemap


Import libraries

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for this first time or if you are getting an authentication error.  

In [2]:
# Authenticates and initializes Earth Engine
import ee
#ee.Authenticate()
ee.Initialize()

## ‘Hello world!’
Printing out information to the console is a basic task for getting information about an object, displaying the numeric result of a computation, displaying object metadata or helping with debugging. The iconic ‘Hello World!’ example is:

In [3]:
# traditional python string
print('Hello world!')

Hello world!


In [4]:
# Earth Eninge object
print(ee.String('Hello World from Vietnam!').getInfo())

Hello World from Vietnam!


## Create an interactive map 
This step creates an interactive map. The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.add_basemap()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [3]:
Map = geemap.Map(center=[21.5,105], zoom= 10)
Map

Map(center=[21.5, 105], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value…

## Adding data to the map
In addition to printing information to the console, adding data to the Map is the way to visualize geographic data. Use Map.addLayer() to do that. In the following example, an Image is instantiated (how to find these images is covered later) using ee.Image(), added to the map with Map.addLayer() and the map is centered over the image:

In [4]:
# define area of interest and time range 
area_Int = ee.Geometry.Rectangle(104.5, 21, 105.5, 22)
start = ee.Date('2020-01-01')
finish = ee.Date('2020-05-01')

# Import the Sentinel 2 collection as sentinel 2
sentinel2 = ee.ImageCollection("COPERNICUS/S2");

Map = geemap.Map(center=[21.5,105], zoom= 9)
Map.add_basemap('OpenTopoMap')
Map.add_basemap('HYBRID')
Map.addLayer(area_Int, {}, 'Area of Interest');
Map

Map(center=[21.5, 105], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value…

In [5]:
# Filter the image collection using filterBounds() and filterDate() method.
# Sort the collection by cloud cover metadata
# Create the mosaic image and clip it to the area of interest

image_dry = sentinel2.filterBounds(area_Int) \
 .filterDate('2020-01-01', '2020-04-30') \
 .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
 .mosaic() \
 .clip(area_Int)
  
image_wet = sentinel2.filterDate('2020-06-01', '2020-10-31') \
 .filterBounds(area_Int) \
 .sort('CLOUDY_PIXEL_PERCENTAGE', False) \
 .mosaic() \
 .clip(area_Int)
  

# Palettes for Visualization
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1500};



## Display Earth Engine data layers 

In [6]:
# Display Earth Engine True Color Images layers in Dry Season

Map = geemap.Map(center=[21.5,105], zoom= 9)
Map.addLayer(image_dry, trueColor_palette, '2020 Dry season true color');


# Also display Earth Engine True Color Images layers in Wet Season

Map.addLayer(image_wet, trueColor_palette, '2020 Wet season true color');
Map

Map(center=[21.5, 105], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value…

## Display NDVI 
Display the resulting index maps
NDVI for Sentinel 2 = (NIR-Red) / (NIR+Red)
NIR = band 8
Red = band 4
Useful for detection of greeness of vegetation or bare areas

In [7]:
# Compute NDVI.

ndvi_dry = image_dry.normalizedDifference(['B8', 'B4']);
ndvi_wet = image_wet.normalizedDifference(['B8', 'B4']);

# Display Earth Engine NDVI Image layer with a color palette for the 2020 Dry Season and Wet Seaason


vis_params = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

Map2 = geemap.Map(center=[21.5,105], zoom= 9)

Map2.addLayer(ndvi_dry, vis_params,  "NDVI_dry")
Map2.addLayer(ndvi_wet, vis_params, "NDVI_wet")
Map2.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")
Map2

Map(center=[21.5, 105], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value…

## Display NDWI 
Display the resulting index maps NDWI for Sentinel 2 = (Green-NIR) / (Green+NIR) Green = band 3 NIR = band 8 - useful for detection of water bodies

In [8]:
# Display Earth Engine NDWI Image layer with a color palette for the 2020 Dry and Wet Seasons

# Calculate NDWI                 
ndwi_dry = image_dry.normalizedDifference(['B3', 'B8']);
ndwi_wet = image_wet.normalizedDifference(['B3', 'B8']);

ndwi_Palette = ['red', 'yellow', 'green', 'blue'];
vis_params = {
  'min': -1,
  'max': 0.5,
  'palette': ndwi_Palette}


Map1 = geemap.Map(center=[21.5,105], zoom= 9)
Map1.addLayer(ndwi_dry, vis_params, '2019 Dry season NDWI');
Map1.addLayer(ndwi_wet, vis_params, '2019 Wet season NDWI');
Map1.add_colorbar(vis_params, label="NDWI", layer_name="NDWI")
Map1

Map(center=[21.5, 105], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value…